## Day 85 Lecture 1 Assignment

In this assignment, we will learn how to use the other layers to improve our model performance.

In [0]:
import numpy as np
import pandas as pd

We will explore a dataset containing information about twitter users and will detect whether or not the user is a bot.

In [0]:
twitter = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/training_data_2_csv_UTF.csv')

In [0]:
twitter.head()

,id,id_str,screen_name,location,description,url,followers_count,friends_count,listed_count,created_at,favourites_count,verified,statuses_count,lang,status,default_profile,default_profile_image,has_extended_profile,name,bot
0,8.160000e+17,"""815745789754417152""","""HoustonPokeMap""","""Houston, TX""","""Rare and strong PokŽmon in Houston, TX. See m...","""https://t.co/dnWuDbFRkt""",1291,0,10,"""Mon Jan 02 02:25:26 +0000 2017""",0,False,78554,"""en""","{\r ""created_at"": ""Sun Mar 12 15:44:04 +0...",True,False,False,"""Houston PokŽ Alert""",1
1,4.843621e+09,4843621225,kernyeahx,"Templeville town, MD, USA",From late 2014 Socium Marketplace will make sh...,NaN,1,349,0,2/1/2016 7:37,38,False,31,en,NaN,True,False,False,Keri Nelson,1
2,4.303727e+09,4303727112,mattlieberisbot,NaN,"Inspired by the smart, funny folks at @replyal...",https://t.co/P1e1o0m4KC,1086,0,14,Fri Nov 20 18:53:22 +0000 2015,0,False,713,en,"{'retweeted': False, 'is_quote_status': False,...",True,False,False,Matt Lieber Is Bot,1
3,3.063139e+09,3063139353,sc_papers,NaN,NaN,NaN,33,0,8,2/25/2015 20:11,0,False,676,en,Construction of human anti-tetanus single-chai...,True,True,False,single cell papers,1
4,2.955142e+09,2955142070,lucarivera16,"Dublin, United States",Inspiring cooks everywhere since 1956.,NaN,11,745,0,1/1/2015 17:44,146,False,185,en,NaN,False,False,False,lucarivera16,1


In [0]:
twitter.bot.value_counts()

0    1476
1    1321
Name: bot, dtype: int64

In [0]:
twitter.isnull().sum()/(twitter.isnull().sum()+twitter.count())

id                       0.000000
id_str                   0.000000
screen_name              0.000000
location                 0.364676
description              0.144083
url                      0.479800
followers_count          0.000000
friends_count            0.000000
listed_count             0.000000
created_at               0.000000
favourites_count         0.000000
verified                 0.000000
statuses_count           0.000000
lang                     0.000000
status                   0.103325
default_profile          0.000000
default_profile_image    0.000000
has_extended_profile     0.035395
name                     0.000000
bot                      0.000000
dtype: float64

In [0]:
for column in twitter.columns:
  if twitter[column].isna().sum()/twitter.shape[0] > .3:
    twitter.drop(columns=[column], inplace=True)

In [0]:
twitter.shape

(2797, 18)

Start by getting rid of all columns that are not useful.

In [0]:
# Answer below:
twitter.drop(columns=['id', 'id_str', 'created_at'], inplace=True)


Next, get rid of all columns that contain more than 30% missing data. After that, remove all rows containing at least one missing observation.

In [0]:
# Answer below:
# twitter.drop(columns=['location', 'url',], inplace=True)


In [0]:
twitter.drop(columns=['status'], inplace=True)

In [0]:
twitter.dropna(inplace=True)

In [0]:
twitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2355 entries, 0 to 2796
Data columns (total 14 columns):
screen_name              2355 non-null object
description              2355 non-null object
followers_count          2355 non-null int64
friends_count            2355 non-null int64
listed_count             2355 non-null int64
favourites_count         2355 non-null int64
verified                 2355 non-null bool
statuses_count           2355 non-null int64
lang                     2355 non-null object
default_profile          2355 non-null bool
default_profile_image    2355 non-null bool
has_extended_profile     2355 non-null object
name                     2355 non-null object
bot                      2355 non-null int64
dtypes: bool(3), int64(6), object(5)
memory usage: 227.7+ KB


Now we will use our embedding functions from a previous assignment.

In [0]:
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words)       

def stem_list(word_list):
    stemmed = []
    for word in word_list:
        stemmedword = stemmer.stem(word)
        stemmed.append(stemmedword)
    return stemmed

def normalize(terms):
    terms = terms.lower()
    terms = remove_stopwords(terms)
    word_delimiters = u'[\\[\\]\n.!?,;:\t\\-\\"\\(\\)\\\'\u2019\u2013 ]'
    term_list = re.split(word_delimiters, terms)
    trimmed = [x.rstrip() for x in term_list]
    stemmed = stem_list(trimmed)
    space = ' '
    normed = space.join(stemmed)
    normed = normed.replace('  ', ' ')
    return normed

We will create two branches, one branch will process the text data in the description and the other will process all other columns. First, create a numpy array with the encoded data from the description column. Normalize each description, one hot encode the text, pad the row and create a numpy array.

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
twitter['normalized'] = twitter.description.apply(normalize)

In [0]:
vocab_size = len(set(twitter.normalized.str.split().sum()))
vocab_size

8020

In [0]:
# vocab_size = np.sum(twitter.normalized.str.len())
twitter['encoded'] = twitter.normalized.apply(one_hot, args=[vocab_size])

In [0]:
independent_vars = pad_sequences(twitter.encoded.values)
independent_vars

array([[   0,    0,    0, ..., 4751, 6587, 3502],
       [   0,    0,    0, ..., 5062, 6490, 7222],
       [   0,    0,    0, ..., 5367, 1898,  870],
       ...,
       [   0,    0,    0, ...,    0,    0, 3403],
       [   0,    0,    0, ..., 2668, 5934, 4467],
       [   0,    0,    0, ..., 6020, 4684, 5225]], dtype=int32)

Convert all boolean variables to numeric (zero for false and 1 for true)

In [0]:
twitter.info()

In [0]:
# Answer below:
twitter[['verified', 'default_profile', 'default_profile_image', 'has_extended_profile']] = twitter[['verified', 'default_profile', 'default_profile_image', 'has_extended_profile']].astype(int)


Create dummy variables out of the relevant object columns. Take caution when converting columns that may incorrectly classified as object.

In [0]:
dummy = pd.get_dummies(twitter['lang'], drop_first=True)
twitter = pd.concat([twitter, dummy], axis=1)
twitter.drop(columns=['lang'], inplace=True)

In [0]:
# Answer below:
twitter.head()


In [0]:
twitter.drop(columns=['normalized', 'encoded'], inplace=True)

In [0]:
twitter.drop(columns=['screen_name', 'name'], inplace=True)

In [0]:
twitter.drop(columns=['description'], inplace=True)

In [0]:
twitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2355 entries, 0 to 2796
Data columns (total 37 columns):
followers_count          2355 non-null int64
friends_count            2355 non-null int64
listed_count             2355 non-null int64
favourites_count         2355 non-null int64
verified                 2355 non-null int64
statuses_count           2355 non-null int64
default_profile          2355 non-null int64
default_profile_image    2355 non-null int64
has_extended_profile     2355 non-null int64
bot                      2355 non-null int64
"en"                     2355 non-null uint8
"en-gb"                  2355 non-null uint8
"es"                     2355 non-null uint8
"fr"                     2355 non-null uint8
"it"                     2355 non-null uint8
"ja"                     2355 non-null uint8
"ko"                     2355 non-null uint8
"pt"                     2355 non-null uint8
"zh-CN"                  2355 non-null uint8
"zh-cn"                  2355 non-null

Min max scale the data decribing each user (do not min max scale the word embeddings).

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
# Answer below:
X = twitter.drop(columns=['bot'], axis=1)
y = np.asarray(twitter['bot'])
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[1.34030216e-05, 0.00000000e+00, 1.61204260e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.03818912e-08, 3.53975838e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.12747339e-05, 0.00000000e+00, 2.25685964e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.67646404e-01, 7.55723201e-03, 1.09871988e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.66110260e-07, 6.49124747e-05, 1.61204260e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.33488734e-04, 3.12391284e-04, 2.16336117e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

Now we'll create the two branches. Create a model for the numeric data that consists of 3 dense layers. An input layer and two hidden layers of size 32.

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, LSTM

In [0]:
X_scaled.shape, independent_vars.shape

((2355, 36), (2355, 66))

In [0]:
independent_vars.shape

(2355, 66)

In [0]:
X_scaled.shape

(2355, 36)

In [0]:
# Answer below:
input_layer1 = Input(shape=(X_scaled.shape[1],))
densel1 = Dense(32, activation='relu')(input_layer1)
densel2 = Dense(32, activation='relu')(densel1)


Create the second branch of the model using the encoded words. This branch will consist of 4 layers: An input layer, an embedding layer returning data of dimension 100, an LSTM layer of unit size 32 and a dense layer of unit size 32. 

In [0]:
# Answer below:
input_layer2 = Input(shape=(independent_vars.shape[1],))
embed = Embedding(vocab_size, 100, input_length=independent_vars.shape[1])(input_layer2)
lstm = LSTM(32)(embed)
densel21 = Dense(32, activation='relu')(lstm)


Merge the two models using the `concatenate` function (merge the two final dense layers in each branch) and create an output dense layer.

In [0]:
from tensorflow.keras.layers import concatenate

In [0]:
merge = concatenate([densel2, densel21])
output = Dense(1, activation='sigmoid')(merge)

In [0]:
type(output)

tensorflow.python.framework.ops.Tensor

Create a model using the two inputs and the single output and print the summary

In [0]:
# Answer below: 
model = Model(inputs=[input_layer1, input_layer2], outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 66)]         0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 36)]         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 66, 100)      802000      input_12[0][0]                   
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 32)           1184        input_11[0][0]                   
______________________________________________________________________________________________

Compile and fit the model using the appropriate optimizer, loss, and metrics. Train the model for 10 epochs with a batch size of 128.

In [0]:
# Answer below:
 model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])   
 model.fit([X_scaled, independent_vars], y2, batch_size=128, epochs=10)   

Train on 2355 samples
Epoch 1/10
2355/2355 [==============================] - 3s 1ms/sample - loss: 0.6730 - acc: 0.6654
Epoch 2/10
2355/2355 [==============================] - 2s 878us/sample - loss: 0.5986 - acc: 0.8038
Epoch 3/10
2355/2355 [==============================] - 2s 873us/sample - loss: 0.4063 - acc: 0.8854
Epoch 4/10
2355/2355 [==============================] - 2s 860us/sample - loss: 0.2154 - acc: 0.9367
Epoch 5/10
2355/2355 [==============================] - 2s 866us/sample - loss: 0.1230 - acc: 0.9682
Epoch 6/10
2355/2355 [==============================] - 2s 852us/sample - loss: 0.0755 - acc: 0.9817
Epoch 7/10
2355/2355 [==============================] - 2s 865us/sample - loss: 0.0545 - acc: 0.9864
Epoch 8/10
2355/2355 [==============================] - 2s 869us/sample - loss: 0.0433 - acc: 0.9902
Epoch 9/10
2355/2355 [==============================] - 2s 860us/sample - loss: 0.0339 - acc: 0.9919
Epoch 10/10
2355/2355 [==============================] - 2s 849us/sampl

In [0]:
y2 = y.reshape(y.shape[0], 1)

In [0]:
y2

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [0]:
X_scaled.shape, y2.shape

((2355, 36), (2355, 1))